In [6]:
import torch
import torch.nn as nn
import torch.nn.utils.parametrize as parametrize

class PositiveWeightParametrization(nn.Module):
    def __init__(self):
        super(PositiveWeightParametrization, self).__init__()

    def forward(self, weight):
        return torch.abs(weight)  # Ensure weights are positive by taking the absolute value


In [7]:
class LargerModel(nn.Module):
    def __init__(self):
        super(LargerModel, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [8]:
# Initialize the model
model = LargerModel()

# Register the positive weight parameterization for each linear layer
parametrize.register_parametrization(model.fc1, "weight", PositiveWeightParametrization())
parametrize.register_parametrization(model.fc2, "weight", PositiveWeightParametrization())
parametrize.register_parametrization(model.fc3, "weight", PositiveWeightParametrization())
parametrize.register_parametrization(model.fc4, "weight", PositiveWeightParametrization())
parametrize.register_parametrization(model.fc5, "weight", PositiveWeightParametrization())


ParametrizedLinear(
  in_features=128, out_features=10, bias=True
  (parametrizations): ModuleDict(
    (weight): ParametrizationList(
      (0): PositiveWeightParametrization()
    )
  )
)

In [9]:
# Example input
x = torch.randn(5, 128)  # Batch of 5 samples, each with 128 features

# Forward pass
output = model(x)

print("\nOutput of the model:")
print(output)



Output of the model:
tensor([[  27.9941,   27.2058,   30.0709,   31.2678,   26.0386,   27.4388,
           27.5907,   28.5043,   27.9966,   25.6480],
        [ 663.8464,  642.5631,  710.6545,  739.7053,  615.9739,  648.6669,
          651.1768,  672.2945,  661.3011,  607.4319],
        [ 244.9572,  237.1760,  262.2976,  272.9989,  227.3346,  239.4125,
          240.3699,  248.1767,  244.0919,  224.1627],
        [  58.0053,   56.2498,   62.1939,   64.7053,   53.8828,   56.7589,
           57.0240,   58.8898,   57.8883,   53.1073],
        [1922.6731, 1860.8177, 2058.0427, 2142.2400, 1783.9044, 1878.5455,
         1885.7285, 1946.8411, 1915.0891, 1759.2177]],
       grad_fn=<AddmmBackward0>)


In [10]:
# Verify that the weights are positive
def check_positive_weights(layer):
    weights = layer.weight
    return torch.all(weights >= 0)

print("\nWeights of fc1 are positive:", check_positive_weights(model.fc1))
print("Weights of fc2 are positive:", check_positive_weights(model.fc2))
print("Weights of fc3 are positive:", check_positive_weights(model.fc3))
print("Weights of fc4 are positive:", check_positive_weights(model.fc4))
print("Weights of fc5 are positive:", check_positive_weights(model.fc5))



Weights of fc1 are positive: tensor(True)
Weights of fc2 are positive: tensor(True)
Weights of fc3 are positive: tensor(True)
Weights of fc4 are positive: tensor(True)
Weights of fc5 are positive: tensor(True)


In [11]:
import torch
import torch.nn as nn

class LargerModelWithoutParam(nn.Module):
    def __init__(self):
        super(LargerModelWithoutParam, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [12]:
def apply_positive_weights(model):
    for layer in model.children():
        if isinstance(layer, nn.Linear):
            with torch.no_grad():
                layer.weight.copy_(torch.abs(layer.weight))

# Initialize the model
model_without_param = LargerModelWithoutParam()

# Apply positive weights manually
apply_positive_weights(model_without_param)


In [13]:
# Example input
x = torch.randn(5, 128)  # Batch of 5 samples, each with 128 features

# Forward pass
output = model_without_param(x)

print("\nOutput of the model:")
print(output)



Output of the model:
tensor([[4309.2729, 4071.0974, 4248.9531, 3724.5676, 3976.8716, 3858.1816,
         4260.5449, 4196.9556, 4418.8042, 4062.3809],
        [3692.2959, 3488.2073, 3640.6135, 3191.3105, 3407.4746, 3305.7998,
         3650.5405, 3596.0508, 3786.1482, 3480.7605],
        [ 856.6445,  809.2256,  844.6451,  740.4554,  790.5018,  767.0203,
          846.9155,  834.2922,  878.4243,  807.6210],
        [ 940.3215,  888.2793,  927.1459,  812.7800,  867.7210,  841.9325,
          929.6431,  915.7894,  964.2242,  886.5009],
        [3580.0078, 3382.1211, 3529.8967, 3094.2617, 3303.8496, 3205.2664,
         3539.5215, 3486.6902, 3671.0056, 3374.9111]],
       grad_fn=<AddmmBackward0>)


In [14]:
# Verify that the weights are positive
def check_positive_weights(layer):
    weights = layer.weight
    return torch.all(weights >= 0)

print("\nWeights of fc1 are positive:", check_positive_weights(model_without_param.fc1))
print("Weights of fc2 are positive:", check_positive_weights(model_without_param.fc2))
print("Weights of fc3 are positive:", check_positive_weights(model_without_param.fc3))
print("Weights of fc4 are positive:", check_positive_weights(model_without_param.fc4))
print("Weights of fc5 are positive:", check_positive_weights(model_without_param.fc5))



Weights of fc1 are positive: tensor(True)
Weights of fc2 are positive: tensor(True)
Weights of fc3 are positive: tensor(True)
Weights of fc4 are positive: tensor(True)
Weights of fc5 are positive: tensor(True)


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Define the models
class LargerModelWithoutParam(nn.Module):
    def __init__(self):
        super(LargerModelWithoutParam, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

class PositiveWeightParametrization(nn.Module):
    def forward(self, X):
        return torch.abs(X)

def apply_positive_weights(model):
    for layer in model.children():
        if isinstance(layer, nn.Linear):
            with torch.no_grad():
                layer.weight.copy_(torch.abs(layer.weight))

# Initialize models
model_without_param = LargerModelWithoutParam()

# Apply positive weights manually
apply_positive_weights(model_without_param)

# Generate random weights for visualization
def get_weight_distributions(model):
    weights = []
    for layer in model.children():
        if isinstance(layer, nn.Linear):
            weights.append(layer.weight.detach().cpu().numpy().flatten())
    return weights

# Get weight distributions
weights_before = get_weight_distributions(model_without_param)

# Apply positive weights again for a visualization
apply_positive_weights(model_without_param)
weights_after = get_weight_distributions(model_without_param)

# Plotting
plt.figure(figsize=(12, 6))

# Plot before constraint
plt.subplot(1, 2, 1)
for weight in weights_before:
    plt.hist(weight, bins=50, alpha=0.6)
plt.title('Weight Distribution Before Constraint')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')

# Plot after constraint
plt.subplot(1, 2, 2)
for weight in weights_after:
    plt.hist(weight, bins=50, alpha=0.6)
plt.title('Weight Distribution After Constraint')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()
